In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import sys
import argparse
import h5py
from scipy.stats import t as student_t
from statsmodels.stats import multitest as mt
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
vars_df = pd.read_pickle('female_vars_ensembles_testset.pkl')

with open('varsHeader.txt') as f:
    varsHeader = f.readlines()
    varsHeader = [l.strip('\n\r') for l in varsHeader]
    varsHeader = np.array(varsHeader)
vars_categories = np.loadtxt("vars_categories.txt", dtype=str, delimiter='\n')

In [3]:
vars_df

,modality,bonf,fdr,dataframe
0,NaiveFull,5.509606,3.300325,idx ...
1,LinearRegressionFull,5.509606,3.500416,idx ...
2,ElasticNetFull,5.509606,3.468050,idx ...
3,Cluster1,5.509606,3.476424,idx ...
4,Cluster2,5.509606,3.506630,idx ...
5,Cluster3,5.509606,3.327015,idx ...
6,Cluster4,5.509606,3.698217,idx ...
7,Cluster5,5.509606,3.631729,idx ...
8,Cluster6,5.509606,3.424072,idx ...
9,Cluster7,5.509606,3.276632,idx ...


In [4]:
vars_df.iloc[0].dataframe

,idx,names,Categories,pearson_r,t_test_statistic,p_values_corrected,p_values,abs_pearson_r,log_p_values,log_p_values_corrected
0,0,Ethnic background (0.0),Ethnic Background,-0.014876,0.744494,1.000000,0.456647,0.014876,0.340419,4.821637e-17
1,1,Ethnic background (1.0),Ethnic Background,-0.028788,0.595809,1.000000,0.551618,0.028788,0.258362,4.821637e-17
2,2,Ethnic background (2.0),Ethnic Background,-0.043779,1.176664,0.972427,0.239718,0.043779,0.620300,1.214292e-02
3,3,Genotype measurement batch (0.0),Genetic Markers,-0.059825,2.966517,0.163519,0.003041,0.059825,2.516975,7.864327e-01
4,4,Heterozygosity (0.0),Genetic Markers,-0.009512,0.470858,1.000000,0.637784,0.009512,0.195326,4.821637e-17
...,...,...,...,...,...,...,...,...,...,...
16160,17519,Methods of self-harm used (0.2),Mental Health,-0.202199,1.130846,0.989572,0.267080,0.202199,0.573359,4.552740e-03
16161,17521,Actions taken following self-harm (0.1),Mental Health,-0.194885,1.577092,0.849418,0.119783,0.194885,0.921605,7.087845e-02
16162,17522,Actions taken following self-harm (0.2),Mental Health,0.009416,0.049826,1.000000,0.960615,0.009416,0.017451,4.821637e-17
16163,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.023469,1.053540,1.000000,0.292220,0.023469,0.534290,4.821637e-17


In [5]:
current_mods = vars_df.modality.values
modality_names = [
    'Naive Average', 'Linear Regression', 'ElasticNet', 'Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5', 'Cluster 6',
    'Cluster 7', 'Cluster 8', 'Cluster 9', 'Cluster 10', 'Cluster 11', 'RENT', 'MLP'
]

for idx, mod in enumerate(current_mods):
    vars_df.loc[vars_df["modality"] == mod, "modality"] = modality_names[idx]
    

In [6]:
# idxs = [1, 2, 14, 15]
# # idxs = [3,4,5,6,7,8,9,10,11,12,13]

ordered_cols = [
     'Modality',
     'bonf',
     'passes_bonf',
     'fdr',
     'passes_fdr',
     'idx',
     'names',
     'Categories',
     'pearson_r',
     'abs_pearson_r',
     't_test_statistic',
     'p_values',
     'p_values_corrected',
     'log_p_values',
     'log_p_values_corrected', 
]

# for idx in idxs:
#     if idx == idxs[0]:
for idx in range(len(vars_df)):
    if idx == 0:
        df = vars_df.iloc[idx].dataframe
        df['Modality'] = vars_df.iloc[idx].modality
        df['bonf'] = vars_df.iloc[idx].bonf
        df['fdr'] = vars_df.iloc[idx].fdr
        df['passes_bonf'] = df.log_p_values > df.bonf
        df['passes_fdr'] = df.log_p_values > df.fdr
        df = df[ordered_cols]
        df = df.drop(columns=['abs_pearson_r'])
    else:
        df2 = vars_df.iloc[idx].dataframe
        df2['Modality'] = vars_df.iloc[idx].modality
        df2['bonf'] = vars_df.iloc[idx].bonf
        df2['fdr'] = vars_df.iloc[idx].fdr
        df2['passes_bonf'] = df2.log_p_values > df2.bonf
        df2['passes_fdr'] = df2.log_p_values > df2.fdr
        df2 = df2[ordered_cols]
        df2 = df2.drop(columns=['abs_pearson_r'])
        df = pd.concat([df, df2])
        del df2

In [7]:
# categories = sorted(list(set(df.Categories.to_list())))

# first_df = True

# for cat in categories:
#     df_tmp = df[(df.Categories==cat) & (df.passes_bonf==True)].sort_values('log_p_values', ascending=False)
#     df_tmp = df_tmp.head(10)
#     if len(df_tmp) == 0:
#         continue
#     if first_df == True:
#         df_print = df_tmp
#         first_df = False
#     else:
#         df_print = pd.concat([df_print, df_tmp])
        
# df_print = df_print.drop(columns=[
#     'bonf', 'passes_bonf', 'fdr', 'passes_fdr', 'idx', 't_test_statistic', 'p_values',
#     'p_values_corrected', 'log_p_values_corrected'
# ])

# df_print_colums_order = [
#     'Modality', 'log_p_values', 'pearson_r', 'Categories', 'names'
# ]

# df_print = df_print[df_print_colums_order]

# df_print = df_print.round(decimals=3)

# df_print = df_print.rename(
#     columns={
#         'log_p_values' : '-log(p)',
#         'pearson_r' : 'Correlation (r)',
#         'Categories': 'Variable Category',
#         'names' : 'Variable Description'
#     }
# )

# df_print



In [8]:
# df_print.to_csv('Famale_top_nIDPs.csv', index=False)

In [9]:
df

,Modality,bonf,passes_bonf,fdr,passes_fdr,idx,names,Categories,pearson_r,t_test_statistic,p_values,p_values_corrected,log_p_values,log_p_values_corrected
0,Naive Average,5.509606,False,3.300325,False,0,Ethnic background (0.0),Ethnic Background,-0.014876,0.744494,0.456647,1.000000,0.340419,4.821637e-17
1,Naive Average,5.509606,False,3.300325,False,1,Ethnic background (1.0),Ethnic Background,-0.028788,0.595809,0.551618,1.000000,0.258362,4.821637e-17
2,Naive Average,5.509606,False,3.300325,False,2,Ethnic background (2.0),Ethnic Background,-0.043779,1.176664,0.239718,0.972427,0.620300,1.214292e-02
3,Naive Average,5.509606,False,3.300325,False,3,Genotype measurement batch (0.0),Genetic Markers,-0.059825,2.966517,0.003041,0.163519,2.516975,7.864327e-01
4,Naive Average,5.509606,False,3.300325,False,4,Heterozygosity (0.0),Genetic Markers,-0.009512,0.470858,0.637784,1.000000,0.195326,4.821637e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16160,MLP,5.509606,False,3.488162,False,17519,Methods of self-harm used (0.2),Mental Health,-0.230031,1.294651,0.205317,0.963411,0.687575,1.618847e-02
16161,MLP,5.509606,False,3.488162,False,17521,Actions taken following self-harm (0.1),Mental Health,-0.276674,2.285240,0.025675,0.594619,1.590482,2.257614e-01
16162,MLP,5.509606,False,3.488162,False,17522,Actions taken following self-harm (0.2),Mental Health,-0.500035,3.055338,0.004896,0.289114,2.310120,5.389315e-01
16163,MLP,5.509606,False,3.488162,False,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.001703,0.076415,0.939097,1.000000,0.027290,-0.000000e+00


In [10]:
df = df.rename(
    columns={
        'log_p_values' : '-log(P)',
        'pearson_r' : 'Correlation (r)',
        'Categories': 'Variable Category',
        'names' : 'Variable Description',
        'bonf': 'Bonferroni Threshold',
        'passes_bonf': 'Passes Bonf.',
        'fdr': 'False Discovery Rate (FDR)',
        'passes_fdr': 'Passes FDR',
        'idx': 'Caterogy ID',
        't_test_statistic': 'T-Values',
        'p_values': 'P-Values',
        'p_values_corrected': 'FDR Adjusted P-Value',
        'log_p_values_corrected': 'FDR Adjusted -log(P)'
    }
)

df

,Modality,Bonferroni Threshold,Passes Bonf.,False Discovery Rate (FDR),Passes FDR,Caterogy ID,Variable Description,Variable Category,Correlation (r),T-Values,P-Values,FDR Adjusted P-Value,-log(P),FDR Adjusted -log(P)
0,Naive Average,5.509606,False,3.300325,False,0,Ethnic background (0.0),Ethnic Background,-0.014876,0.744494,0.456647,1.000000,0.340419,4.821637e-17
1,Naive Average,5.509606,False,3.300325,False,1,Ethnic background (1.0),Ethnic Background,-0.028788,0.595809,0.551618,1.000000,0.258362,4.821637e-17
2,Naive Average,5.509606,False,3.300325,False,2,Ethnic background (2.0),Ethnic Background,-0.043779,1.176664,0.239718,0.972427,0.620300,1.214292e-02
3,Naive Average,5.509606,False,3.300325,False,3,Genotype measurement batch (0.0),Genetic Markers,-0.059825,2.966517,0.003041,0.163519,2.516975,7.864327e-01
4,Naive Average,5.509606,False,3.300325,False,4,Heterozygosity (0.0),Genetic Markers,-0.009512,0.470858,0.637784,1.000000,0.195326,4.821637e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16160,MLP,5.509606,False,3.488162,False,17519,Methods of self-harm used (0.2),Mental Health,-0.230031,1.294651,0.205317,0.963411,0.687575,1.618847e-02
16161,MLP,5.509606,False,3.488162,False,17521,Actions taken following self-harm (0.1),Mental Health,-0.276674,2.285240,0.025675,0.594619,1.590482,2.257614e-01
16162,MLP,5.509606,False,3.488162,False,17522,Actions taken following self-harm (0.2),Mental Health,-0.500035,3.055338,0.004896,0.289114,2.310120,5.389315e-01
16163,MLP,5.509606,False,3.488162,False,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.001703,0.076415,0.939097,1.000000,0.027290,-0.000000e+00


In [11]:
df.to_csv('Famale_ALL_nIDPs_ENSEMBLES.csv', index=False)

In [12]:
df_passing_FDR = df[df['Passes FDR']==True]

In [13]:
df_passing_FDR

,Modality,Bonferroni Threshold,Passes Bonf.,False Discovery Rate (FDR),Passes FDR,Caterogy ID,Variable Description,Variable Category,Correlation (r),T-Values,P-Values,FDR Adjusted P-Value,-log(P),FDR Adjusted -log(P)
262,Naive Average,5.509606,False,3.300325,True,283,Age at menopause (last menstrual period) (2.0),Lifestyle,-0.084525,3.665329,0.000254,0.026482,3.595299,1.577055
956,Naive Average,5.509606,False,3.300325,True,1172,Coffee intake (3.0),Diet,0.359461,3.694788,0.000373,0.036585,3.427788,1.436697
1566,Naive Average,5.509606,False,3.300325,True,1908,Crispbread intake (3.0),Diet,0.137881,4.041895,0.000058,0.007141,4.237532,2.146228
1601,Naive Average,5.509606,False,3.300325,True,1943,Number of crackers/crispbreads with butter/mar...,Diet,0.145734,3.571992,0.000383,0.037101,3.416475,1.430616
2398,Naive Average,5.509606,False,3.300325,True,2778,Alcohol intake frequency. (0.0),Alcohol,0.086238,4.334959,0.000015,0.002058,4.819479,2.686450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14778,MLP,5.509606,False,3.488162,True,16109,Operative procedures - OPCS4 (A842 - A84.2 Ele...,Medical History,0.072359,3.633996,0.000285,0.041457,3.545660,1.382407
15593,MLP,5.509606,False,3.488162,True,16926,Operative procedures - OPCS4 (U055 - U05.5 Mag...,Medical History,0.082557,4.149441,0.000034,0.007329,4.462955,2.134979
15635,MLP,5.509606,False,3.488162,True,16968,Operative procedures - OPCS4 (V256 - V25.6 Pri...,Medical History,0.076724,3.854485,0.000119,0.020727,3.924873,1.683462
15851,MLP,5.509606,False,3.488162,True,17184,Operative procedures - OPCS4 (X893 - X89.3 Imm...,Medical History,0.075042,3.769486,0.000167,0.025765,3.776355,1.588969


In [14]:
df_passing_FDR.to_csv('Famale_ALL_nIDPs_passing_FDR_ENSEMBLES.csv', index=False)